In [2]:
from data_prep import my_sheet
import pandas as pd
import altair as alt

In [3]:
data = my_sheet[0].batch_get(['A1:Z1000'])[0]
df = pd.DataFrame(data, columns=data.pop(0))

my_teammates = my_sheet[4].batch_get(['A1:Z10000'])[0]
teammates_df = pd.DataFrame(my_teammates, columns=my_teammates.pop(0))

In [4]:
df

,Season,Date,Club,Team,Competition,Stage,Opposition,Home/Away,Venue,Result,F,A,Position,Time,Start,MOTM,YC,Try,Notes,Opposition_club
0,2015/2016,2015-11-21,Ealing,2nd,MMT 2,NA,Rosslyn Park IVs (4th),A,Away,L,5,40,Lock,80,TRUE,FALSE,FALSE,0,NA,Rosslyn Park IVs
1,2015/2016,2015-11-28,Ealing,Exiles,MMT Vase,NA,Uxbridge 2nd,A,Away,L,0,17,No. 8,80,TRUE,FALSE,FALSE,0,NA,Uxbridge
2,2015/2016,2015-12-05,Ealing,Exiles,MMT 5,NA,Twickenham Bulldogs (2nd),H,Ealing,L,15,29,Lock,80,TRUE,FALSE,FALSE,0,NA,Twickenham Bulldogs
3,2015/2016,2015-12-12,Ealing,Exiles,MMT 5,NA,Hammersmith & Fulham 5th,A,Away,L,26,38,Lock,80,TRUE,FALSE,FALSE,0,NA,Hammersmith & Fulham
4,2015/2016,2015-12-19,Ealing,Exiles,Friendly,NA,High Wycombe 3rd,A,Away,W,12,5,Lock,80,TRUE,FALSE,FALSE,0,NA,High Wycombe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2023/2024,2024-02-17,East Grinstead,1st,Counties 1 Surrey/Sussex,,Teddington,A,Away,L,0,26,Lock,60,TRUE,FALSE,FALSE,0,None,None
172,2023/2024,2024-03-02,East Grinstead,1st,Counties 1 Surrey/Sussex,,Eastbourne,H,Home,W,15,14,Lock,80,TRUE,FALSE,FALSE,0,None,None
173,2023/2024,2024-03-23,East Grinstead,1st,Counties 1 Surrey/Sussex,,Hove,H,Home,L,13,15,Lock,80,TRUE,FALSE,FALSE,0,None,None
174,2023/2024,2024-04-06,East Grinstead,1st,Counties 1 Surrey/Sussex,,Twickenham,A,Away,L,10,35,Lock,80,TRUE,TRUE,FALSE,1,None,None


# Teammates

In [5]:
teammates_df

,Club,Team,name,Date,cap,pos
0,East Grinstead,1st,Guy Collins,2024-04-13,FALSE,1
1,East Grinstead,1st,Ben Tottman,2024-04-13,FALSE,2
2,East Grinstead,1st,Josh Brimecombe,2024-04-13,FALSE,3
3,East Grinstead,1st,Dan Billin,2024-04-13,FALSE,4
4,East Grinstead,1st,Sam Lindsay-McCall,2024-04-13,FALSE,5
...,...,...,...,...,...,...
3333,Ealing,2nd,Nick Yau,2015-11-21,FALSE,14
3334,Ealing,2nd,Eoghan O'Sullivan,2015-11-21,FALSE,15
3335,Ealing,2nd,Sam Lindsay,2015-11-21,FALSE,16
3336,Ealing,2nd,Harry Charalanbous,2015-11-21,FALSE,17


In [102]:
tdf = teammates_df[~teammates_df['name'].isin(['Sam Lindsay', 'Sam Lindsay-McCall'])]
tdf.rename(columns={"name":"Player"}, inplace=True)

# Position dictionary
d = {
    1: "Prop",
    2: "Hooker",
    3: "Prop",
    4: "Second Row",
    5: "Second Row",
    6: "Back Row",
    7: "Back Row",
    8: "Back Row",
    9: "Scrum Half",
    10: "Fly Half",
    11: "Back Three",
    12: "Centre",
    13: "Centre",
    14: "Back Three",
    15: "Back Three",
}

# Top 3 players by pos
tdf['pos'] = pd.to_numeric(tdf["pos"],errors='coerce')
tdf["Position"] = tdf["pos"].apply(lambda x: d[x] if x in d.keys() else None)

counts = df.value_counts('Club').to_dict()

alt.Chart(tdf).mark_bar().encode(
    y=alt.Y('Player:N', sort='-x', title=None),
    x=alt.X('countX:Q', title='No. of games'),
    color=alt.Color(
        'Club:N', 
        scale=alt.Scale(domain=["Ealing", "East Grinstead"], range=['darkgreen', 'darkblue']), 
        legend=alt.Legend(title=None, orient='bottom-right', fillColor="white", labelFontSize=12, symbolSize=150, padding=10, strokeColor="black"),
    ),
    tooltip=['Player', 'Club', alt.Tooltip('count(Player)', title="Games")]
).transform_aggregate(
    countX='count()',
    groupby=['Player', 'Club']
).transform_filter(
    "datum.countX > 30"
).properties(
    title=alt.Title(
        text='Top Teammates',
        subtitle=[
            f"Out of {counts['Ealing']} games for Ealing", f"and {counts['East Grinstead']} games for East Grinstead"],
        fontSize=24,
        offset=20,
    ),
    width=400,
)

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_10011/1969150160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf.rename(columns={"name":"Player"}, inplace=True)
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_10011/1969150160.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['pos'] = pd.to_numeric(tdf["pos"],errors='coerce')
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_10011/1969150160.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

alt.Chart(...)

In [108]:
tdf.groupby('Position').apply(lambda x: x.value_counts("Player").head(3)).reset_index()

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_10011/1395983051.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tdf.groupby('Position').apply(lambda x: x.value_counts("Player").head(3)).reset_index()


,Position,Player,count
0,Back Row,Aaron Boczek,28
1,Back Row,Ryland Thomas,27
2,Back Row,Abdou Traore,20
3,Back Three,Jake Radcliffe,37
4,Back Three,Ali Moffatt,28
5,Back Three,Sam Spriddell,21
6,Centre,Max Crawley-Moore,44
7,Centre,Ryan Morlen,15
8,Centre,Ryan Walsh,15
9,Fly Half,Dan Poulton,38


In [109]:
df

,Season,Date,Club,Team,Competition,Stage,Opposition,Home/Away,Venue,Result,F,A,Position,Time,Start,MOTM,YC,Try,Notes,Opposition_club
0,2015/2016,2015-11-21,Ealing,2nd,MMT 2,NA,Rosslyn Park IVs (4th),A,Away,L,5,40,Lock,80,TRUE,FALSE,FALSE,0,NA,Rosslyn Park IVs
1,2015/2016,2015-11-28,Ealing,Exiles,MMT Vase,NA,Uxbridge 2nd,A,Away,L,0,17,No. 8,80,TRUE,FALSE,FALSE,0,NA,Uxbridge
2,2015/2016,2015-12-05,Ealing,Exiles,MMT 5,NA,Twickenham Bulldogs (2nd),H,Ealing,L,15,29,Lock,80,TRUE,FALSE,FALSE,0,NA,Twickenham Bulldogs
3,2015/2016,2015-12-12,Ealing,Exiles,MMT 5,NA,Hammersmith & Fulham 5th,A,Away,L,26,38,Lock,80,TRUE,FALSE,FALSE,0,NA,Hammersmith & Fulham
4,2015/2016,2015-12-19,Ealing,Exiles,Friendly,NA,High Wycombe 3rd,A,Away,W,12,5,Lock,80,TRUE,FALSE,FALSE,0,NA,High Wycombe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2023/2024,2024-02-17,East Grinstead,1st,Counties 1 Surrey/Sussex,,Teddington,A,Away,L,0,26,Lock,60,TRUE,FALSE,FALSE,0,None,None
172,2023/2024,2024-03-02,East Grinstead,1st,Counties 1 Surrey/Sussex,,Eastbourne,H,Home,W,15,14,Lock,80,TRUE,FALSE,FALSE,0,None,None
173,2023/2024,2024-03-23,East Grinstead,1st,Counties 1 Surrey/Sussex,,Hove,H,Home,L,13,15,Lock,80,TRUE,FALSE,FALSE,0,None,None
174,2023/2024,2024-04-06,East Grinstead,1st,Counties 1 Surrey/Sussex,,Twickenham,A,Away,L,10,35,Lock,80,TRUE,TRUE,FALSE,1,None,None


In [179]:
mins = alt.Chart(df).mark_line(
    color="black", 
    opacity=0.8, 
    point=alt.OverlayMarkDef(filled=True, fill="black", size=50)
).encode(
    x = alt.X('Season:N', axis=alt.Axis(labelAngle=-60, titleFontSize=16, labelFontSize=12)),
    y = alt.Y("sum(Time):Q", title="Minutes played", axis=alt.Axis(orient='right', titleFontSize=14), scale=alt.Scale(domain=[0, 2000])),
)

base = alt.Chart(df).encode(
    x = alt.X('Season:N', axis=alt.Axis(labelAngle=-60)),
    y = alt.Y("count():Q", title="Games played", axis=alt.Axis(orient='left', titleFontSize=14), scale=alt.Scale(domain=[0, 40])),
)
games = base.mark_bar(opacity=0.8).encode(
    color=alt.Color(
        'Club:N', 
        scale=alt.Scale(domain=["Ealing", "East Grinstead"], range=['darkgreen', 'darkblue']), 
        legend=None
    ),
)
games_text = base.mark_text(fontSize=15, dy=-10, ).encode(
    text = alt.Text("count():Q", title="Minutes played"),
)


(
    (games + games_text) + mins
).resolve_scale(y='independent').properties(
    title=alt.Title(
        text='Games / Minutes per season',
        fontSize=24,
        offset=20,
    ),
    width=400,
)

alt.LayerChart(...)

In [200]:
base = alt.Chart(df).encode(
    x = alt.X('Season:N', axis=alt.Axis(labelAngle=-30)),
    y = alt.Y("sum(Try):Q", title="Tries scored", axis=alt.Axis(orient='left', titleFontSize=14, format=".0f")),
)

tries = base.mark_bar(opacity=0.8).encode(
    color=alt.Color(
        'Club:N', 
        scale=alt.Scale(domain=["Ealing", "East Grinstead"], range=['darkgreen', 'darkblue']), 
        legend=None
    ),
)
tries_text = base.mark_text(fontSize=15, dy=-10, ).encode(
    text = alt.Text("average(Try):Q", title="Tries scored", format=".2f"),
)
(tries + tries_text).properties(width=alt.Step(50), title=alt.Title(
        text='Tries per season',
        subtitle="Label shows average tries per game",
        fontSize=24,
        offset=20,
    ))

alt.LayerChart(...)

In [226]:
# Split position column into two columns (pos1 and pos2) by splitting on "/"
df[['pos1', 'pos2']] = df['Position'].str.split("/", expand=True)

# melt the dataframe to have a single column for position
df_melt = pd.melt(df, id_vars=['Season', 'Club'], value_vars=['pos1', 'pos2'], value_name='pos').dropna()

alt.Chart(df_melt).mark_bar().encode(
    y=alt.Y('pos:N', title=None, sort=["Lock", "Flanker", "No. 8"]),
    x=alt.X('count()', title="Games"),
    row=alt.Row('Season', title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelFontStyle="bold"), spacing=10),
    color=alt.Color('Club:N', scale=alt.Scale(domain=["Ealing", "East Grinstead"], range=['darkgreen', 'darkblue']), legend=None),
).resolve_scale(y='independent').properties(
    title=alt.Title(
        text='Games played by position',
        fontSize=24,
        offset=20,
        anchor='middle',
    ),
    width=300,
)

alt.Chart(...)

In [264]:
alt.Chart(df).mark_bar(width=3).encode(
    y=alt.Y('F:Q', title='Points for and against', scale=alt.Scale(nice=False)),
    y2=alt.Y2('A:Q'),
    x=alt.X('Date:T', scale=alt.Scale(nice=False)),
    color=alt.Color('Result:N', 
        scale=alt.Scale(domain=["L", "W"], range=['red','darkgreen']), 
        legend=alt.Legend(orient='top-left', fillColor="white", labelFontSize=12, symbolSize=150, padding=10, strokeColor="black", direction="horizontal"),),
    tooltip=['Season', 'Club', 'Team', 'Result', 'F', 'A'],
).properties(
    title=alt.Title(
        text='Points difference per game',
        fontSize=24,
        offset=20,
        anchor='middle',
    ),
    width=1000,
)

alt.Chart(...)

In [231]:
df

,Season,Date,Club,Team,Competition,Stage,Opposition,Home/Away,Venue,Result,...,Position,Time,Start,MOTM,YC,Try,Notes,Opposition_club,pos1,pos2
0,2015/2016,2015-11-21,Ealing,2nd,MMT 2,NA,Rosslyn Park IVs (4th),A,Away,L,...,Lock,80,TRUE,FALSE,FALSE,0,NA,Rosslyn Park IVs,Lock,None
1,2015/2016,2015-11-28,Ealing,Exiles,MMT Vase,NA,Uxbridge 2nd,A,Away,L,...,No. 8,80,TRUE,FALSE,FALSE,0,NA,Uxbridge,No. 8,None
2,2015/2016,2015-12-05,Ealing,Exiles,MMT 5,NA,Twickenham Bulldogs (2nd),H,Ealing,L,...,Lock,80,TRUE,FALSE,FALSE,0,NA,Twickenham Bulldogs,Lock,None
3,2015/2016,2015-12-12,Ealing,Exiles,MMT 5,NA,Hammersmith & Fulham 5th,A,Away,L,...,Lock,80,TRUE,FALSE,FALSE,0,NA,Hammersmith & Fulham,Lock,None
4,2015/2016,2015-12-19,Ealing,Exiles,Friendly,NA,High Wycombe 3rd,A,Away,W,...,Lock,80,TRUE,FALSE,FALSE,0,NA,High Wycombe,Lock,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2023/2024,2024-02-17,East Grinstead,1st,Counties 1 Surrey/Sussex,,Teddington,A,Away,L,...,Lock,60,TRUE,FALSE,FALSE,0,None,None,Lock,None
172,2023/2024,2024-03-02,East Grinstead,1st,Counties 1 Surrey/Sussex,,Eastbourne,H,Home,W,...,Lock,80,TRUE,FALSE,FALSE,0,None,None,Lock,None
173,2023/2024,2024-03-23,East Grinstead,1st,Counties 1 Surrey/Sussex,,Hove,H,Home,L,...,Lock,80,TRUE,FALSE,FALSE,0,None,None,Lock,None
174,2023/2024,2024-04-06,East Grinstead,1st,Counties 1 Surrey/Sussex,,Twickenham,A,Away,L,...,Lock,80,TRUE,TRUE,FALSE,1,None,None,Lock,None
